## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,78.17,76,13,16.73,light rain
1,1,Dikson,RU,73.5069,80.5464,-23.39,100,30,6.78,scattered clouds
2,2,Ribeira Grande,PT,38.5167,-28.7000,62.60,72,40,13.80,scattered clouds
3,3,Mataura,NZ,-46.1927,168.8643,53.01,97,100,2.68,overcast clouds
4,4,Marsabit,KE,2.3284,37.9899,75.74,39,100,12.35,overcast clouds


In [3]:
# Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [4]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,78.17,76,13,16.73,light rain
4,4,Marsabit,KE,2.3284,37.9899,75.74,39,100,12.35,overcast clouds
17,17,Dingle,PH,10.9995,122.6711,76.86,86,80,5.95,broken clouds
18,18,Richards Bay,ZA,-28.7830,32.0377,70.38,70,96,10.63,overcast clouds
29,29,Thai Binh,VN,20.4500,106.3333,73.40,94,90,3.44,overcast clouds
32,32,Cape Town,ZA,-33.9258,18.4232,75.99,46,0,11.50,clear sky
40,40,Rawson,AR,-43.3002,-65.1023,70.65,41,78,10.94,broken clouds
48,48,Vallenar,CL,-28.5708,-70.7581,75.96,40,0,5.46,clear sky
49,49,Chuy,UY,-33.6971,-53.4616,70.65,81,72,8.50,broken clouds
51,51,Vaini,TO,-21.2000,-175.2000,77.00,100,75,8.05,light rain


In [5]:
# Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,78.17,76,13,16.73,light rain
4,4,Marsabit,KE,2.3284,37.9899,75.74,39,100,12.35,overcast clouds
17,17,Dingle,PH,10.9995,122.6711,76.86,86,80,5.95,broken clouds
18,18,Richards Bay,ZA,-28.7830,32.0377,70.38,70,96,10.63,overcast clouds
29,29,Thai Binh,VN,20.4500,106.3333,73.40,94,90,3.44,overcast clouds
32,32,Cape Town,ZA,-33.9258,18.4232,75.99,46,0,11.50,clear sky
40,40,Rawson,AR,-43.3002,-65.1023,70.65,41,78,10.94,broken clouds
48,48,Vallenar,CL,-28.5708,-70.7581,75.96,40,0,5.46,clear sky
49,49,Chuy,UY,-33.6971,-53.4616,70.65,81,72,8.50,broken clouds
51,51,Vaini,TO,-21.2000,-175.2000,77.00,100,75,8.05,light rain


In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.17,light rain,-23.1203,-134.9692,
4,Marsabit,KE,75.74,overcast clouds,2.3284,37.9899,
17,Dingle,PH,76.86,broken clouds,10.9995,122.6711,
18,Richards Bay,ZA,70.38,overcast clouds,-28.7830,32.0377,
29,Thai Binh,VN,73.40,overcast clouds,20.4500,106.3333,
32,Cape Town,ZA,75.99,clear sky,-33.9258,18.4232,
40,Rawson,AR,70.65,broken clouds,-43.3002,-65.1023,
48,Vallenar,CL,75.96,clear sky,-28.5708,-70.7581,
49,Chuy,UY,70.65,broken clouds,-33.6971,-53.4616,
51,Vaini,TO,77.00,light rain,-21.2000,-175.2000,


In [8]:
# Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# Drop any rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.17,light rain,-23.1203,-134.9692,Pension Maro'i
4,Marsabit,KE,75.74,overcast clouds,2.3284,37.9899,Chicho Hotel
17,Dingle,PH,76.86,broken clouds,10.9995,122.6711,SEGAYA PROPERTY
18,Richards Bay,ZA,70.38,overcast clouds,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
29,Thai Binh,VN,73.40,overcast clouds,20.4500,106.3333,Sweden Hotel
32,Cape Town,ZA,75.99,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
40,Rawson,AR,70.65,broken clouds,-43.3002,-65.1023,Hosteria Sampedro
48,Vallenar,CL,75.96,clear sky,-28.5708,-70.7581,Humacao Bed & Breakfast
49,Chuy,UY,70.65,broken clouds,-33.6971,-53.4616,Nuevo Hotel Plaza
51,Vaini,TO,77.00,light rain,-21.2000,-175.2000,Keleti Beach Resort


In [10]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))